In [ ]:
%%capture
!pip install pytorch-tabnet

In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
import joblib
import pickle
import json

import torch
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import json
import os


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [ ]:
X_train_csv = pd.read_csv("/content/drive/MyDrive/GR-II/CARVEME/50/X_train_GR-II.csv")
X_val_csv = pd.read_csv("/content/drive/MyDrive/GR-II/CARVEME/50/X_val_GR-II.csv")
X_test_csv = pd.read_csv("/content/drive/MyDrive/GR-II/CARVEME/50/X_test_GR-II.csv")
y_train_csv = pd.read_csv("/content/drive/MyDrive/GR-II/CARVEME/50/y_train_GR-II.csv")
y_val_csv = pd.read_csv("/content/drive/MyDrive/GR-II/CARVEME/50/y_val_GR-II.csv")
y_test_csv = pd.read_csv("/content/drive/MyDrive/GR-II/CARVEME/50/y_test_GR-II.csv")

In [ ]:
def to_npy(X_train_csv, X_val_csv, X_test_csv, y_train_csv, y_val_csv, y_test_csv):
    X_train = X_train_csv.to_numpy()
    X_val = X_val_csv.to_numpy()
    X_test = X_test_csv.to_numpy()
    y_train = y_train_csv.to_numpy()
    y_val = y_val_csv.to_numpy()
    y_test = y_test_csv.to_numpy()

    #
    np.save("X_train.npy", X_train)
    np.save("X_val.npy", X_val)
    np.save("X_test.npy", X_test)
    np.save("y_train.npy", y_train)
    np.save("y_val.npy", y_val)
    np.save("y_test.npy", y_test)

    print("CSV onverted to .npy format.")

In [ ]:
to_npy(X_train_csv, X_val_csv, X_test_csv, y_train_csv, y_val_csv, y_test_csv)

CSV onverted to .npy format.


In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = np.load("X_train.npy"), np.load("y_train.npy"), np.load("X_val.npy"), np.load("y_val.npy"), np.load("X_test.npy"), np.load("y_test.npy")

In [ ]:
def train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test,
                          output_dir="tabnet_regression_results",
                          max_epochs=20, patience=20, seed=42):
    """
    Complete TabNet regression pipeline with GPU support and evaluation

    Args:
        X_train, y_train: Training data (numpy arrays)
        X_val, y_val: Validation data
        X_test, y_test: Test data
        output_dir: Directory to save results
        max_epochs: Maximum training epochs
        patience: Early stopping patience
        seed: seed
    Returns:
        Dictionary with test metrics and trained model
    """

    os.makedirs(output_dir, exist_ok=True)

    # Set
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Init
    regressor = TabNetRegressor(
        cat_idxs=[],
        cat_dims=[],
        cat_emb_dim=1,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
        scheduler_params={"step_size":50, "gamma":0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type='entmax',
        n_d=64,
        n_a=64,
        n_steps=5,
        gamma=1.3,
        n_independent=2,
        n_shared=2,
        lambda_sparse=1e-4,
        seed=seed,
        device_name=device,
        verbose=1
    )

    regressor.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_val, y_val)],
        eval_name=['val'],
        eval_metric=['rmse'],  # RMSE
        max_epochs=max_epochs,
        patience=patience,
        batch_size=1024,
        virtual_batch_size=128,
        num_workers=0,
        drop_last=False
    )

    # Make predictions
    y_train_pred = regressor.predict(X_train)
    y_val_pred = regressor.predict(X_val)
    y_test_pred = regressor.predict(X_test)

    # Calculate metrics
    def calculate_metrics(y_true, y_pred):
        return {
            "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
            "MAE": mean_absolute_error(y_true, y_pred),
            "R2": r2_score(y_true, y_pred),
            "MSE": mean_squared_error(y_true, y_pred)
        }

    metrics = {
        "train": calculate_metrics(y_train, y_train_pred),
        "val": calculate_metrics(y_val, y_val_pred),
        "test": calculate_metrics(y_test, y_test_pred)
    }

    # Save results
    with open(os.path.join(output_dir, "metrics.json"), 'w') as f:
        json.dump(metrics, f, indent=4)

    # Save model
    model_path = os.path.join(output_dir, "tabnet_regressor.zip")
    regressor.save_model(model_path)
    print(f"Model saved to {model_path}")

    # Print summary
    print("\nTest Set Performance:")
    for metric, value in metrics['test'].items():
        print(f"{metric}: {value:.4f}")

    return {"model": regressor, "metrics": metrics}

#AGORA-GR-I-100

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1970.42552| val_rmse: 26.66686|  0:00:06s
epoch 1  | loss: 571.60392| val_rmse: 22.39707|  0:00:11s
epoch 2  | loss: 466.91346| val_rmse: 22.20339|  0:00:15s
epoch 3  | loss: 396.49823| val_rmse: 19.46165|  0:00:20s
epoch 4  | loss: 351.93216| val_rmse: 18.7779 |  0:00:25s
epoch 5  | loss: 322.24825| val_rmse: 17.76201|  0:00:30s
epoch 6  | loss: 290.29037| val_rmse: 17.08634|  0:00:35s
epoch 7  | loss: 263.5121| val_rmse: 16.89906|  0:00:40s
epoch 8  | loss: 240.54349| val_rmse: 16.80437|  0:00:45s
epoch 9  | loss: 221.86834| val_rmse: 16.84593|  0:00:49s
epoch 10 | loss: 203.49396| val_rmse: 16.94339|  0:00:54s
epoch 11 | loss: 193.36666| val_rmse: 16.85305|  0:00:59s
epoch 12 | loss: 180.22711| val_rmse: 16.72937|  0:01:04s
epoch 13 | loss: 162.77947| val_rmse: 16.98213|  0:01:09s
epoch 14 | loss: 152.94528| val_rmse: 16.87823|  0:01:13s
epoch 15 | loss: 141.40531| val_rmse: 17.4604 |  0:01:18s
epoch 16 | loss: 135.12298| val_rmse: 17.20184|  0:01:23s
epoch 17 | los

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_regression_results/tabnet_regressor.zip.zip
Model saved to tabnet_regression_results/tabnet_regressor.zip

Test Set Performance:
RMSE: 16.9279
MAE: 11.3039
R2: 0.6765
MSE: 286.5527


{'model': TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.0001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02, 'weight_decay': 1e-05}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'step_size': 50, 'gamma': 0.9}, mask_type='entmax', input_dim=424, output_dim=1, device_name='cuda', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]),
 'metrics': {'train': {'RMSE': 11.624746466499273,
   'MAE': 8.153693367703744,
   'R2': 0.8475804829486777,
   'MSE': 135.13473041038733},
  'val': {'RMSE': 16.729374036251453,
   'MAE': 11.186195672489509,
   'R2': 0.6813364294325983,
   'MSE': 279.8719556448042},
  'test': {'RMSE': 16.92786874346203,
   'MAE': 11.303913452931932,
   'R2': 0.6764980618223053,
   'MSE': 286.5527401958788}}}

#AGORA-GR-II-100

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1178.78141| val_rmse: 23.96364|  0:00:10s
epoch 1  | loss: 451.54849| val_rmse: 20.08675|  0:00:20s
epoch 2  | loss: 349.91803| val_rmse: 17.97423|  0:00:30s
epoch 3  | loss: 294.00827| val_rmse: 16.95351|  0:00:41s
epoch 4  | loss: 263.65732| val_rmse: 16.66498|  0:00:51s
epoch 5  | loss: 244.43441| val_rmse: 16.23092|  0:01:01s
epoch 6  | loss: 226.2207| val_rmse: 16.08054|  0:01:11s
epoch 7  | loss: 211.20506| val_rmse: 16.19622|  0:01:22s
epoch 8  | loss: 198.05539| val_rmse: 15.97059|  0:01:32s
epoch 9  | loss: 189.29643| val_rmse: 16.20921|  0:01:42s
epoch 10 | loss: 179.7184| val_rmse: 16.11902|  0:01:53s
epoch 11 | loss: 172.22265| val_rmse: 16.32338|  0:02:03s
epoch 12 | loss: 162.11871| val_rmse: 16.3129 |  0:02:13s
epoch 13 | loss: 159.72863| val_rmse: 16.16807|  0:02:23s
epoch 14 | loss: 148.9762| val_rmse: 16.18306|  0:02:34s
epoch 15 | loss: 139.48127| val_rmse: 16.37623|  0:02:44s
epoch 16 | loss: 136.79602| val_rmse: 16.56429|  0:02:54s
epoch 17 | loss:

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_regression_results/tabnet_regressor.zip.zip
Model saved to tabnet_regression_results/tabnet_regressor.zip

Test Set Performance:
RMSE: 15.9393
MAE: 10.9807
R2: 0.6751
MSE: 254.0622


{'model': TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.0001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02, 'weight_decay': 1e-05}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'step_size': 50, 'gamma': 0.9}, mask_type='entmax', input_dim=424, output_dim=1, device_name='cuda', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]),
 'metrics': {'train': {'RMSE': 12.857704600140982,
   'MAE': 9.071877671813892,
   'R2': 0.7864035433083123,
   'MSE': 165.32056758448655},
  'val': {'RMSE': 15.970590567168044,
   'MAE': 11.079483394238514,
   'R2': 0.667270554780292,
   'MSE': 255.0597630641169},
  'test': {'RMSE': 15.939329461320007,
   'MAE': 10.980674198615905,
   'R2': 0.675093145050538,
   'MSE': 254.06222367650395}}}

#AGORA-GR-III-100

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 990.75074| val_rmse: 20.52218|  0:00:18s
epoch 1  | loss: 340.72604| val_rmse: 16.85107|  0:00:37s
epoch 2  | loss: 274.04345| val_rmse: 16.11558|  0:00:56s
epoch 3  | loss: 247.74007| val_rmse: 15.64781|  0:01:15s
epoch 4  | loss: 233.04525| val_rmse: 15.5401 |  0:01:34s
epoch 5  | loss: 222.44864| val_rmse: 15.38397|  0:01:52s
epoch 6  | loss: 214.00588| val_rmse: 15.29415|  0:02:11s
epoch 7  | loss: 206.37686| val_rmse: 15.33953|  0:02:30s
epoch 8  | loss: 203.29956| val_rmse: 15.47932|  0:02:49s
epoch 9  | loss: 195.94197| val_rmse: 15.51663|  0:03:07s
epoch 10 | loss: 191.60317| val_rmse: 15.52653|  0:03:26s
epoch 11 | loss: 186.71765| val_rmse: 15.36665|  0:03:45s
epoch 12 | loss: 181.7833| val_rmse: 15.69875|  0:04:03s
epoch 13 | loss: 177.99101| val_rmse: 15.59022|  0:04:22s
epoch 14 | loss: 176.64001| val_rmse: 15.50416|  0:04:41s
epoch 15 | loss: 174.3726| val_rmse: 15.66487|  0:05:00s
epoch 16 | loss: 169.93622| val_rmse: 15.61696|  0:05:18s
epoch 17 | loss:

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_regression_results/tabnet_regressor.zip.zip
Model saved to tabnet_regression_results/tabnet_regressor.zip

Test Set Performance:
RMSE: 15.3048
MAE: 10.9732
R2: 0.7450
MSE: 234.2355


{'model': TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.0001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02, 'weight_decay': 1e-05}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'step_size': 50, 'gamma': 0.9}, mask_type='entmax', input_dim=424, output_dim=1, device_name='cuda', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]),
 'metrics': {'train': {'RMSE': 13.859981198652841,
   'MAE': 10.061658499130653,
   'R2': 0.792251053525696,
   'MSE': 192.09907882701026},
  'val': {'RMSE': 15.294148672663315,
   'MAE': 10.982996747136806,
   'R2': 0.7462912601876908,
   'MSE': 233.91098362152903},
  'test': {'RMSE': 15.30475327340917,
   'MAE': 10.973194478673678,
   'R2': 0.7450423531444254,
   'MSE': 234.2354727599287}}}

#AGORA-GR-I-50

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 4774.34322| val_rmse: 68.26354|  0:00:00s
epoch 1  | loss: 1863.69465| val_rmse: 46.6047 |  0:00:01s
epoch 2  | loss: 884.71458| val_rmse: 38.07274|  0:00:02s
epoch 3  | loss: 699.20463| val_rmse: 29.73602|  0:00:03s
epoch 4  | loss: 644.41055| val_rmse: 27.32586|  0:00:03s
epoch 5  | loss: 607.56345| val_rmse: 26.27285|  0:00:04s
epoch 6  | loss: 578.44064| val_rmse: 25.57018|  0:00:05s
epoch 7  | loss: 555.68542| val_rmse: 24.78933|  0:00:06s
epoch 8  | loss: 532.19121| val_rmse: 24.21022|  0:00:07s
epoch 9  | loss: 522.13596| val_rmse: 24.00497|  0:00:08s
epoch 10 | loss: 503.07536| val_rmse: 23.55965|  0:00:08s
epoch 11 | loss: 489.32307| val_rmse: 23.7052 |  0:00:09s
epoch 12 | loss: 467.0699| val_rmse: 23.63557|  0:00:10s
epoch 13 | loss: 450.02393| val_rmse: 23.18981|  0:00:11s
epoch 14 | loss: 434.6808| val_rmse: 23.69625|  0:00:11s
epoch 15 | loss: 414.90679| val_rmse: 22.28143|  0:00:12s
epoch 16 | loss: 392.65911| val_rmse: 22.19682|  0:00:13s
epoch 17 | los

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_regression_results/tabnet_regressor.zip.zip
Model saved to tabnet_regression_results/tabnet_regressor.zip

Test Set Performance:
RMSE: 21.3581
MAE: 16.2435
R2: 0.3320
MSE: 456.1670


{'model': TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.0001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02, 'weight_decay': 1e-05}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'step_size': 50, 'gamma': 0.9}, mask_type='entmax', input_dim=424, output_dim=1, device_name='cuda', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]),
 'metrics': {'train': {'RMSE': 19.501286973897695,
   'MAE': 14.996283359224803,
   'R2': 0.45463296060510916,
   'MSE': 380.3001936383119},
  'val': {'RMSE': 21.73352633896387,
   'MAE': 16.595166182298353,
   'R2': 0.3342902050331753,
   'MSE': 472.3461671264363},
  'test': {'RMSE': 21.35806596898982,
   'MAE': 16.243471773690985,
   'R2': 0.3320375858203194,
   'MSE': 456.166981935721}}}

#AGORA-GR-II-50

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1357.48912| val_rmse: 26.41925|  0:00:03s
epoch 1  | loss: 542.6576| val_rmse: 23.13621|  0:00:06s
epoch 2  | loss: 506.33892| val_rmse: 22.38535|  0:00:10s
epoch 3  | loss: 460.14514| val_rmse: 21.16278|  0:00:13s
epoch 4  | loss: 416.54343| val_rmse: 20.66094|  0:00:16s
epoch 5  | loss: 393.81416| val_rmse: 19.96618|  0:00:20s
epoch 6  | loss: 370.47901| val_rmse: 19.75915|  0:00:23s
epoch 7  | loss: 352.19662| val_rmse: 19.44905|  0:00:27s
epoch 8  | loss: 339.96742| val_rmse: 19.19114|  0:00:30s
epoch 9  | loss: 319.18629| val_rmse: 19.41992|  0:00:33s
epoch 10 | loss: 300.00305| val_rmse: 19.17536|  0:00:37s
epoch 11 | loss: 280.38276| val_rmse: 19.13271|  0:00:40s
epoch 12 | loss: 252.46379| val_rmse: 19.00484|  0:00:43s
epoch 13 | loss: 235.2966| val_rmse: 19.56943|  0:00:47s
epoch 14 | loss: 212.80943| val_rmse: 19.26859|  0:00:50s
epoch 15 | loss: 196.60608| val_rmse: 19.70539|  0:00:53s
epoch 16 | loss: 180.84496| val_rmse: 19.73361|  0:00:57s
epoch 17 | loss

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_regression_results/tabnet_regressor.zip.zip
Model saved to tabnet_regression_results/tabnet_regressor.zip

Test Set Performance:
RMSE: 18.9728
MAE: 14.1889
R2: 0.3577
MSE: 359.9660


{'model': TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.0001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02, 'weight_decay': 1e-05}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'step_size': 50, 'gamma': 0.9}, mask_type='entmax', input_dim=424, output_dim=1, device_name='cuda', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]),
 'metrics': {'train': {'RMSE': 14.64314234223926,
   'MAE': 10.989982547177988,
   'R2': 0.6161199709793281,
   'MSE': 214.4216176550803},
  'val': {'RMSE': 19.004835404312217,
   'MAE': 14.2072800572981,
   'R2': 0.35660349287287074,
   'MSE': 361.1837687449991},
  'test': {'RMSE': 18.97277090365879,
   'MAE': 14.188941969361622,
   'R2': 0.3576717165605108,
   'MSE': 359.9660357627217}}}

#AGORA-GR-III-50

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 694.65239| val_rmse: 19.78122|  0:00:21s
epoch 1  | loss: 359.59432| val_rmse: 18.13342|  0:00:43s
epoch 2  | loss: 315.23477| val_rmse: 18.14707|  0:01:05s
epoch 3  | loss: 293.93335| val_rmse: 17.31864|  0:01:27s
epoch 4  | loss: 279.44327| val_rmse: 17.07191|  0:01:50s
epoch 5  | loss: 267.08676| val_rmse: 17.09724|  0:02:11s
epoch 6  | loss: 257.64613| val_rmse: 17.23477|  0:02:33s
epoch 7  | loss: 251.08143| val_rmse: 17.05524|  0:02:55s
epoch 8  | loss: 242.72793| val_rmse: 17.22549|  0:03:17s
epoch 9  | loss: 236.0007| val_rmse: 17.07636|  0:03:39s
epoch 10 | loss: 231.37094| val_rmse: 17.1513 |  0:04:01s
epoch 11 | loss: 226.43055| val_rmse: 17.21141|  0:04:23s
epoch 12 | loss: 220.10295| val_rmse: 17.24351|  0:04:45s
epoch 13 | loss: 214.6977| val_rmse: 17.82638|  0:05:07s
epoch 14 | loss: 209.24656| val_rmse: 17.4701 |  0:05:29s
epoch 15 | loss: 203.49546| val_rmse: 17.4994 |  0:05:51s
epoch 16 | loss: 205.28905| val_rmse: 17.4695 |  0:06:13s
epoch 17 | loss:

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_regression_results/tabnet_regressor.zip.zip
Model saved to tabnet_regression_results/tabnet_regressor.zip

Test Set Performance:
RMSE: 17.0025
MAE: 12.7253
R2: 0.5755
MSE: 289.0852


{'model': TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.0001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02, 'weight_decay': 1e-05}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'step_size': 50, 'gamma': 0.9}, mask_type='entmax', input_dim=424, output_dim=1, device_name='cuda', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]),
 'metrics': {'train': {'RMSE': 14.98075710175032,
   'MAE': 11.30653094860255,
   'R2': 0.6718045156299893,
   'MSE': 224.42308334164267},
  'val': {'RMSE': 17.055242410122965,
   'MAE': 12.765974744724398,
   'R2': 0.5747158420692962,
   'MSE': 290.88129366805697},
  'test': {'RMSE': 17.002505718107688,
   'MAE': 12.725341662104887,
   'R2': 0.5754561803535903,
   'MSE': 289.0852006942846}}}

#CARVEME-GR-I-100

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 275.91912| val_rmse: 13.72617|  0:00:09s
epoch 1  | loss: 167.44663| val_rmse: 12.77314|  0:00:19s
epoch 2  | loss: 149.74297| val_rmse: 12.04198|  0:00:29s
epoch 3  | loss: 130.63086| val_rmse: 11.25429|  0:00:39s
epoch 4  | loss: 113.45753| val_rmse: 10.57794|  0:00:48s
epoch 5  | loss: 97.9394 | val_rmse: 10.01026|  0:00:58s
epoch 6  | loss: 84.1232 | val_rmse: 11.59939|  0:01:08s
epoch 7  | loss: 73.5883 | val_rmse: 9.50221 |  0:01:18s
epoch 8  | loss: 65.67803| val_rmse: 9.47898 |  0:01:27s
epoch 9  | loss: 60.30728| val_rmse: 9.43986 |  0:01:37s
epoch 10 | loss: 53.48012| val_rmse: 9.5856  |  0:01:47s
epoch 11 | loss: 50.25825| val_rmse: 9.52196 |  0:01:57s
epoch 12 | loss: 48.31548| val_rmse: 9.45645 |  0:02:06s
epoch 13 | loss: 45.02582| val_rmse: 9.5495  |  0:02:16s
epoch 14 | loss: 41.95205| val_rmse: 9.508   |  0:02:26s
epoch 15 | loss: 41.28416| val_rmse: 9.50866 |  0:02:35s
epoch 16 | loss: 39.6803 | val_rmse: 9.72205 |  0:02:46s
epoch 17 | loss: 35.88905|

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_regression_results/tabnet_regressor.zip.zip
Model saved to tabnet_regression_results/tabnet_regressor.zip

Test Set Performance:
RMSE: 9.4187
MAE: 6.9673
R2: 0.5339
MSE: 88.7124


{'model': TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.0001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02, 'weight_decay': 1e-05}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'step_size': 50, 'gamma': 0.9}, mask_type='entmax', input_dim=499, output_dim=1, device_name='cuda', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]),
 'metrics': {'train': {'RMSE': 6.837368759802856,
   'MAE': 5.149975211524846,
   'R2': 0.7533273396160711,
   'MSE': 46.74961155752804},
  'val': {'RMSE': 9.439858191359896,
   'MAE': 6.978239923913603,
   'R2': 0.5338962203016145,
   'MSE': 89.11092267298451},
  'test': {'RMSE': 9.418726086598207,
   'MAE': 6.967307085434858,
   'R2': 0.533920089179255,
   'MSE': 88.71240109436557}}}

#CARVEME-GR-II-100

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 309.42325| val_rmse: 14.72722|  0:00:13s
epoch 1  | loss: 164.54752| val_rmse: 12.54759|  0:00:26s
epoch 2  | loss: 137.01939| val_rmse: 11.37533|  0:00:39s
epoch 3  | loss: 113.96649| val_rmse: 10.59458|  0:00:52s
epoch 4  | loss: 96.92002| val_rmse: 10.08793|  0:01:05s
epoch 5  | loss: 85.35332| val_rmse: 9.92534 |  0:01:18s
epoch 6  | loss: 77.17515| val_rmse: 9.74617 |  0:01:31s
epoch 7  | loss: 70.21997| val_rmse: 9.73998 |  0:01:44s
epoch 8  | loss: 66.31061| val_rmse: 9.94777 |  0:01:58s
epoch 9  | loss: 62.2088 | val_rmse: 9.79792 |  0:02:11s
epoch 10 | loss: 58.65605| val_rmse: 9.81905 |  0:02:24s
epoch 11 | loss: 56.55619| val_rmse: 9.83016 |  0:02:37s
epoch 12 | loss: 54.07171| val_rmse: 9.76009 |  0:02:50s
epoch 13 | loss: 50.54027| val_rmse: 9.83946 |  0:03:03s
epoch 14 | loss: 48.35479| val_rmse: 10.05488|  0:03:15s
epoch 15 | loss: 47.28848| val_rmse: 9.88948 |  0:03:28s
epoch 16 | loss: 46.2207 | val_rmse: 9.90546 |  0:03:41s
epoch 17 | loss: 43.95813| 

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_regression_results/tabnet_regressor.zip.zip
Model saved to tabnet_regression_results/tabnet_regressor.zip

Test Set Performance:
RMSE: 9.7750
MAE: 7.2829
R2: 0.6778
MSE: 95.5509


{'model': TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.0001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02, 'weight_decay': 1e-05}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'step_size': 50, 'gamma': 0.9}, mask_type='entmax', input_dim=499, output_dim=1, device_name='cuda', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]),
 'metrics': {'train': {'RMSE': 7.4795958836549366,
   'MAE': 5.608962850041652,
   'R2': 0.8117931367407064,
   'MSE': 55.944354582787874},
  'val': {'RMSE': 9.739976839570003,
   'MAE': 7.269882946191126,
   'R2': 0.6789336465135991,
   'MSE': 94.86714883536006},
  'test': {'RMSE': 9.77501524779249,
   'MAE': 7.2829202062985585,
   'R2': 0.6777848886883626,
   'MSE': 95.55092309457567}}}

#CARVEME-GR-III-100

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 309.42325| val_rmse: 14.72722|  0:00:13s
epoch 1  | loss: 164.54752| val_rmse: 12.54759|  0:00:26s
epoch 2  | loss: 137.01939| val_rmse: 11.37533|  0:00:40s
epoch 3  | loss: 113.96649| val_rmse: 10.59458|  0:00:53s
epoch 4  | loss: 96.92002| val_rmse: 10.08793|  0:01:06s
epoch 5  | loss: 85.35332| val_rmse: 9.92534 |  0:01:19s
epoch 6  | loss: 77.17515| val_rmse: 9.74617 |  0:01:32s
epoch 7  | loss: 70.21997| val_rmse: 9.73998 |  0:01:46s
epoch 8  | loss: 66.31061| val_rmse: 9.94777 |  0:01:59s
epoch 9  | loss: 62.2088 | val_rmse: 9.79792 |  0:02:12s
epoch 10 | loss: 58.65605| val_rmse: 9.81905 |  0:02:24s
epoch 11 | loss: 56.55619| val_rmse: 9.83016 |  0:02:38s
epoch 12 | loss: 54.07171| val_rmse: 9.76009 |  0:02:51s
epoch 13 | loss: 50.54027| val_rmse: 9.83946 |  0:03:04s
epoch 14 | loss: 48.35479| val_rmse: 10.05488|  0:03:17s
epoch 15 | loss: 47.28848| val_rmse: 9.88948 |  0:03:30s
epoch 16 | loss: 46.2207 | val_rmse: 9.90546 |  0:03:43s
epoch 17 | loss: 43.95813| 

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_regression_results/tabnet_regressor.zip.zip
Model saved to tabnet_regression_results/tabnet_regressor.zip

Test Set Performance:
RMSE: 9.7750
MAE: 7.2829
R2: 0.6778
MSE: 95.5509


{'model': TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.0001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02, 'weight_decay': 1e-05}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'step_size': 50, 'gamma': 0.9}, mask_type='entmax', input_dim=499, output_dim=1, device_name='cuda', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]),
 'metrics': {'train': {'RMSE': 7.4795958836549366,
   'MAE': 5.608962850041652,
   'R2': 0.8117931367407064,
   'MSE': 55.944354582787874},
  'val': {'RMSE': 9.739976839570003,
   'MAE': 7.269882946191126,
   'R2': 0.6789336465135991,
   'MSE': 94.86714883536006},
  'test': {'RMSE': 9.77501524779249,
   'MAE': 7.2829202062985585,
   'R2': 0.6777848886883626,
   'MSE': 95.55092309457567}}}

#CARVEME-GR-I-50

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 348.39892| val_rmse: 13.98544|  0:00:04s
epoch 1  | loss: 177.10945| val_rmse: 13.89728|  0:00:08s
epoch 2  | loss: 157.01449| val_rmse: 12.67314|  0:00:13s
epoch 3  | loss: 146.35884| val_rmse: 12.68824|  0:00:18s
epoch 4  | loss: 139.90198| val_rmse: 12.78477|  0:00:22s
epoch 5  | loss: 135.64932| val_rmse: 11.99297|  0:00:27s
epoch 6  | loss: 130.40408| val_rmse: 11.7358 |  0:00:31s
epoch 7  | loss: 126.242 | val_rmse: 11.69754|  0:00:36s
epoch 8  | loss: 117.12462| val_rmse: 11.60712|  0:00:40s
epoch 9  | loss: 110.15625| val_rmse: 11.72884|  0:00:45s
epoch 10 | loss: 102.07907| val_rmse: 11.78433|  0:00:49s
epoch 11 | loss: 93.84025| val_rmse: 11.73735|  0:00:54s
epoch 12 | loss: 87.36083| val_rmse: 11.78335|  0:00:58s
epoch 13 | loss: 76.98274| val_rmse: 11.95256|  0:01:03s
epoch 14 | loss: 67.50783| val_rmse: 11.86559|  0:01:07s
epoch 15 | loss: 63.24046| val_rmse: 12.09907|  0:01:12s
epoch 16 | loss: 56.42278| val_rmse: 12.38774|  0:01:16s
epoch 17 | loss: 50.8

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_regression_results/tabnet_regressor.zip.zip
Model saved to tabnet_regression_results/tabnet_regressor.zip

Test Set Performance:
RMSE: 11.5318
MAE: 9.0647
R2: 0.2805
MSE: 132.9832


{'model': TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.0001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02, 'weight_decay': 1e-05}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'step_size': 50, 'gamma': 0.9}, mask_type='entmax', input_dim=499, output_dim=1, device_name='cuda', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]),
 'metrics': {'train': {'RMSE': 10.275504940573688,
   'MAE': 8.093388692549365,
   'R2': 0.43105671768233234,
   'MSE': 105.58600178375428},
  'val': {'RMSE': 11.607121738656426,
   'MAE': 9.157879716831088,
   'R2': 0.27971594160839586,
   'MSE': 134.72527505599058},
  'test': {'RMSE': 11.531835880158376,
   'MAE': 9.064725102218874,
   'R2': 0.2804917483169286,
   'MSE': 132.9832387669081}}}

#CARVEME-GR-II-50

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 344.66948| val_rmse: 13.87161|  0:00:04s
epoch 1  | loss: 176.11388| val_rmse: 13.42749|  0:00:09s
epoch 2  | loss: 161.07357| val_rmse: 12.88445|  0:00:13s
epoch 3  | loss: 149.87405| val_rmse: 12.6063 |  0:00:18s
epoch 4  | loss: 144.45816| val_rmse: 12.25834|  0:00:22s
epoch 5  | loss: 138.89699| val_rmse: 12.55105|  0:00:27s
epoch 6  | loss: 134.40475| val_rmse: 11.7501 |  0:00:31s
epoch 7  | loss: 128.67169| val_rmse: 11.7219 |  0:00:36s
epoch 8  | loss: 123.32819| val_rmse: 11.6059 |  0:00:41s
epoch 9  | loss: 114.99992| val_rmse: 11.53777|  0:00:45s
epoch 10 | loss: 106.25767| val_rmse: 11.66239|  0:00:49s
epoch 11 | loss: 95.60991| val_rmse: 11.73716|  0:00:54s
epoch 12 | loss: 88.69513| val_rmse: 11.81562|  0:00:59s
epoch 13 | loss: 79.58863| val_rmse: 11.87014|  0:01:03s
epoch 14 | loss: 67.76991| val_rmse: 11.85188|  0:01:08s
epoch 15 | loss: 62.91055| val_rmse: 12.13386|  0:01:12s
epoch 16 | loss: 57.17528| val_rmse: 12.03633|  0:01:17s
epoch 17 | loss: 50.

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_regression_results/tabnet_regressor.zip.zip
Model saved to tabnet_regression_results/tabnet_regressor.zip

Test Set Performance:
RMSE: 11.5405
MAE: 9.0612
R2: 0.2794
MSE: 133.1836


{'model': TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.0001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02, 'weight_decay': 1e-05}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'step_size': 50, 'gamma': 0.9}, mask_type='entmax', input_dim=499, output_dim=1, device_name='cuda', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]),
 'metrics': {'train': {'RMSE': 10.115933868756896,
   'MAE': 7.987403915940201,
   'R2': 0.4485900579722152,
   'MSE': 102.33211803706284},
  'val': {'RMSE': 11.537765132884294,
   'MAE': 9.09957027554718,
   'R2': 0.2882981215775151,
   'MSE': 133.12002426160055},
  'test': {'RMSE': 11.540519084025327,
   'MAE': 9.061191894942832,
   'R2': 0.279407794458967,
   'MSE': 133.1835807287528}}}

#CARVEME-GR-III-50

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 344.66948| val_rmse: 13.87161|  0:00:04s
epoch 1  | loss: 176.11388| val_rmse: 13.42749|  0:00:09s
epoch 2  | loss: 161.07357| val_rmse: 12.88445|  0:00:13s
epoch 3  | loss: 149.87405| val_rmse: 12.6063 |  0:00:18s
epoch 4  | loss: 144.45816| val_rmse: 12.25834|  0:00:22s
epoch 5  | loss: 138.89699| val_rmse: 12.55105|  0:00:27s
epoch 6  | loss: 134.40475| val_rmse: 11.7501 |  0:00:31s
epoch 7  | loss: 128.67169| val_rmse: 11.7219 |  0:00:36s
epoch 8  | loss: 123.32819| val_rmse: 11.6059 |  0:00:41s
epoch 9  | loss: 114.99992| val_rmse: 11.53777|  0:00:45s
epoch 10 | loss: 106.25767| val_rmse: 11.66239|  0:00:50s
epoch 11 | loss: 95.60991| val_rmse: 11.73716|  0:00:54s
epoch 12 | loss: 88.69513| val_rmse: 11.81562|  0:00:59s
epoch 13 | loss: 79.58863| val_rmse: 11.87014|  0:01:03s
epoch 14 | loss: 67.76991| val_rmse: 11.85188|  0:01:08s
epoch 15 | loss: 62.91055| val_rmse: 12.13386|  0:01:13s
epoch 16 | loss: 57.17528| val_rmse: 12.03633|  0:01:17s
epoch 17 | loss: 50.

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_regression_results/tabnet_regressor.zip.zip
Model saved to tabnet_regression_results/tabnet_regressor.zip

Test Set Performance:
RMSE: 11.5405
MAE: 9.0612
R2: 0.2794
MSE: 133.1836


{'model': TabNetRegressor(n_d=64, n_a=64, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.0001, seed=42, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02, 'weight_decay': 1e-05}, scheduler_fn=<class 'torch.optim.lr_scheduler.StepLR'>, scheduler_params={'step_size': 50, 'gamma': 0.9}, mask_type='entmax', input_dim=499, output_dim=1, device_name='cuda', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]),
 'metrics': {'train': {'RMSE': 10.115933868756896,
   'MAE': 7.987403915940201,
   'R2': 0.4485900579722152,
   'MSE': 102.33211803706284},
  'val': {'RMSE': 11.537765132884294,
   'MAE': 9.09957027554718,
   'R2': 0.2882981215775151,
   'MSE': 133.12002426160055},
  'test': {'RMSE': 11.540519084025327,
   'MAE': 9.061191894942832,
   'R2': 0.279407794458967,
   'MSE': 133.1835807287528}}}